In [86]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings("ignore")

In [87]:
USE_DATA_LEAK='Y' ## Set to Y to exploit the data leak
##
RAND_VAL=44
num_folds=7 ## Number of folds
n_est=5000 ## Number of estimators

In [58]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [59]:
print(df_train.shape)
df_train.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [60]:
print(df_test.shape)
df_test.head()

(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [61]:
df_train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [62]:
df_train.drop('Cabin', axis=1, inplace=True)

In [63]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [64]:
from sklearn.impute import SimpleImputer

In [69]:
imputer = SimpleImputer(strategy='mean')

In [72]:
df_train['Age']=imputer.fit_transform(df_train[['Age']])
df_test['Age']=imputer.fit_transform(df_test[['Age']])

In [73]:
df_train['Embarked']=df_train['Embarked'].fillna('S')

In [74]:
print(df_train.isna().sum())
print(df_test.isna().sum())

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64
PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [78]:
imp = SimpleImputer(strategy='most_frequent')
df_test['Cabin'] = imp.fit_transform(df_test[['Cabin']]).reshape(-1)

In [79]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [80]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,B57 B59 B63 B66,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,B57 B59 B63 B66,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,B57 B59 B63 B66,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,B57 B59 B63 B66,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,B57 B59 B63 B66,S


In [83]:
num_tr=df_train.select_dtypes(include=['number']).columns
cat_tr=df_train.select_dtypes(include=['object']).columns
print(cat_tr)
print(num_tr)

Index(['Name', 'Sex', 'Ticket', 'Embarked'], dtype='object')
Index(['PassengerId', 'Survived', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')


In [84]:
num_te=df_test.select_dtypes(include=['number']).columns
cat_te=df_test.select_dtypes(include=['object']).columns
print(cat_te)
print(num_te)

Index(['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], dtype='object')
Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')


In [94]:
df_train['mix'] = df_train['Name'].astype('str')+df_train['Sex']+df_train['Ticket']+df_train['Embarked']
df_test['mix'] = df_test['Name'].astype('str')+df_test['Sex']+df_test['Ticket']+df_test['Embarked']

In [95]:
df_train['mix']

0                  Braund, Mr. Owen HarrismaleA/5 21171S
1      Cumings, Mrs. John Bradley (Florence Briggs Th...
2          Heikkinen, Miss. LainafemaleSTON/O2. 3101282S
3      Futrelle, Mrs. Jacques Heath (Lily May Peel)fe...
4                    Allen, Mr. William Henrymale373450S
                             ...                        
886                     Montvila, Rev. Juozasmale211536S
887            Graham, Miss. Margaret Edithfemale112053S
888    Johnston, Miss. Catherine Helen "Carrie"female...
889                     Behr, Mr. Karl Howellmale111369C
890                       Dooley, Mr. Patrickmale370376Q
Name: mix, Length: 891, dtype: object

In [96]:
def get_vectors(df_train,df_test,col_name):
    vectorizer = TfidfVectorizer(max_features=1000)
    vectors_train = vectorizer.fit_transform(df_train[col_name])
    vectors_test = vectorizer.transform(df_test[col_name])
    #
    svd = TruncatedSVD(3)
    x_pca_train = svd.fit_transform(vectors_train)
    x_pca_test = svd.transform(vectors_test)

    # Convert to DataFrames
    tfidf_df_train = pd.DataFrame(x_pca_train)
    tfidf_df_test = pd.DataFrame(x_pca_test)

    # Naming columns in the new DataFrames
    cols = [(col_name + "_tfidf_" + str(f)) for f in tfidf_df_train.columns.to_list()]
    tfidf_df_train.columns = cols
    tfidf_df_test.columns = cols

    # Reset the index of the DataFrames before concatenation
    df_train = df_train.reset_index(drop=True)
    df_test = df_test.reset_index(drop=True)

    # Concatenate transformed features with original data
    df_train = pd.concat([df_train, tfidf_df_train], axis="columns")
    df_test = pd.concat([df_test, tfidf_df_test], axis="columns")
    return df_train,df_test

In [97]:
df_train,df_test = get_vectors(df_train,df_test,'mix')

In [98]:
df_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,mix,mix_tfidf_0,mix_tfidf_1,mix_tfidf_2,mix_tfidf_0,mix_tfidf_1,mix_tfidf_2
0,892,3,"Kelly, Mr. James",male,34.50000,0,0,330911,7.8292,B57 B59 B63 B66,Q,"Kelly, Mr. Jamesmale330911Q",0.243837,0.007522,0.000019,0.243837,0.007522,0.000608
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.00000,1,0,363272,7.0000,B57 B59 B63 B66,S,"Wilkes, Mrs. James (Ellen Needs)female363272S",0.009663,0.025325,0.185002,0.009663,0.025328,0.184876
2,894,2,"Myles, Mr. Thomas Francis",male,62.00000,0,0,240276,9.6875,B57 B59 B63 B66,Q,"Myles, Mr. Thomas Francismale240276Q",0.311180,0.000173,0.081069,0.311180,0.000174,0.076997
3,895,3,"Wirz, Mr. Albert",male,27.00000,0,0,315154,8.6625,B57 B59 B63 B66,S,"Wirz, Mr. Albertmale315154S",0.994449,-0.003652,-0.054781,0.994449,-0.003652,-0.054630
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.00000,1,1,3101298,12.2875,B57 B59 B63 B66,S,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)fe...",0.003572,0.003926,0.172294,0.003572,0.003925,0.172790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,30.27259,0,0,A.5. 3236,8.0500,B57 B59 B63 B66,S,"Spector, Mr. WoolfmaleA.5. 3236S",0.994449,-0.003652,-0.054781,0.994449,-0.003652,-0.054630
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.00000,0,0,PC 17758,108.9000,C105,C,"Oliva y Ocana, Dona. FerminafemalePC 17758C",0.000000,0.000000,0.000000,0.000723,0.000026,0.003060
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.50000,0,0,SOTON/O.Q. 3101262,7.2500,B57 B59 B63 B66,S,"Saether, Mr. Simon SivertsenmaleSOTON/O.Q. 310...",0.994449,-0.003652,-0.054781,0.994449,-0.003652,-0.054630
416,1308,3,"Ware, Mr. Frederick",male,30.27259,0,0,359309,8.0500,B57 B59 B63 B66,S,"Ware, Mr. Frederickmale359309S",0.994449,-0.003652,-0.054781,0.994449,-0.003652,-0.054630


In [108]:
feat_cols=df_train.drop(['Name', 'Sex', 'Ticket','Embarked'], axis=1).columns
feat_cols=df_test.drop(['Name', 'Sex', 'Ticket','Embarked','Cabin'], axis=1).columns

print(feat_cols)

Index(['PassengerId', 'Pclass', 'Age', 'SibSp', 'Parch', 'Fare', 'mix',
       'mix_tfidf_0', 'mix_tfidf_1', 'mix_tfidf_2', 'mix_tfidf_0',
       'mix_tfidf_1', 'mix_tfidf_2'],
      dtype='object')


In [109]:
X=df_train[feat_cols]
y=df_train['Survived']